In [ ]:
#!pip install -U swifter

In [ ]:
# !pip install -U pandas
# !pip install -U seaborn

In [ ]:
#!sed -i ':a;N;$!ba;s/\,\n/\,/g' /scratch/amq259/nlu-s2021-final-project/results/Cameron*

In [1]:
import re
import pandas as pd
import analysis_relabel_funcs
from analysis import read_dfs, read_outfile
import torch
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from torch.nn import Softmax
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import pickle
import swifter
import time
%matplotlib inline

In [2]:
MODEL_RELABEL = {
    'BERT-Jigsaw': ["not toxic", "toxic"], 
    'BERT-SBIC-offensive': ["Not", "Maybe", "Offensive"],
    'BERT-SBIC-targetcategory': ['none', 'body', 'culture', 'disabled', 'gender', 'race', 'social', 'victim'], # not used
    'BERT-eec-emotion': ['none', 'anger','fear', 'joy', 'sadness'], # not used
    'BERT-jigsaw-identityhate': ['Not', "Yes"],
    'BERT-jigsaw-severetoxic': ["Not", "Yes"],
    'BERT-mdgender-convai-binary': ["female", "male"],
    'BERT-mdgender-convai-ternary': ["female", "male", "neutral"],
    'BERT-mdgender-wizard': ["neutral", "female", "male"],
    'BERT-rtgender-opgender-annotations': ["man", "woman"] 
}

In [3]:
def _softmax_and_relabel(predictions, categorical_labels):
    # takes in a torch.Tensor of predictions and a list of categorical labels, returns a tuple of (softmax tensor, categorical label)
    m = Softmax(dim=0)
    sm = m(predictions)
    return sm, categorical_labels[sm.argmax().item()]

def score(labels, predictions):
    softmax_preds, category = _softmax_and_relabel(predictions, labels)
    # scores are always the difference between the first and last
    
    # for eec-emotion
    if labels[1] == "anger": 
        return softmax_preds[1], category
    
    # for target_category
    if labels[1] == "body":
        return softmax_preds[1], category
    
    return softmax_preds[-1] - softmax_preds[0], category

In [11]:
eval_files = {}
toxic_files = []
gender_files = []

TOXIC_MODELS = frozenset(["Cameron/BERT-Jigsaw", 'Cameron/BERT-SBIC-offensive'])
                #'Cameron/BERT-jigsaw-identityhate', 'Cameron/BERT-jigsaw-severetoxic'
GENDER_MODELS = frozenset([
    "Cameron/BERT-eec-emotion",
    "Cameron/BERT-SBIC-targetcategory",
    "Cameron/BERT-mdgender-convai-binary",
    "Cameron/BERT-mdgender-convai-ternary", 
    "Cameron/BERT-mdgender-wizard", 
    "Cameron/BERT-rtgender-opgender-annotations"])

import glob
raw_files = glob.glob("/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT*")
FILES = []
for f in raw_files: 
    FILES.append(f.replace("/scratch/amq259/nlu-s2021-final-project/", ""))

print(FILES[:3])

def create_tup(filename, model_name):
    return (model_name.replace("Cameron/", ""), filename)

for filename in FILES:
    
    s = filename[len("results/"):].split(":")
    
    model_name = s[0][:-3]
    model_name = model_name.replace("Cameron-", "Cameron/")
    #print(TOXIC_MODELS)
    #print(GENDER_MODELS)
    #print(f"Attempting to classify {model_name}, in toxic {model_name in TOXIC_MODELS}, in gender {model_name in GENDER_MODELS}")
    eval_dataset_name = s[-1][3:-15]
    model_type = None
    if eval_dataset_name not in eval_files:
        eval_files[eval_dataset_name] = {"toxic": [], "gender": []}
    if model_name in TOXIC_MODELS:
        model_type = "TOXIC"
        eval_files[eval_dataset_name]["toxic"].append(create_tup("/scratch/amq259/nlu-s2021-final-project/" + filename, model_name))
        toxic_files.append(filename)
    elif model_name in GENDER_MODELS:
        model_type = "GENDER"
        eval_files[eval_dataset_name]["gender"].append(create_tup("/scratch/amq259/nlu-s2021-final-project/" + filename, model_name))
        gender_files.append(filename)
    else:
        print(f"Model type is not classified, please classify {model_name} with file {filename}")
        continue
    print(f"Model name: {model_name} | eval dataset: {eval_dataset_name}" )
    print("-------------------------")

['results/Cameron-BERT-rtgender-opgender-annotations-02:35:33-wikipedia1114982-train-eval.out', 'results/Cameron-BERT-mdgender-wizard-09:24:57-billsum-train-eval.out', 'results/Cameron-BERT-mdgender-convai-binary-00:42:04-air_dialogue-train-eval.out']
Model name: Cameron/BERT-rtgender-opgender-annotations | eval dataset: wikipedia1114982
-------------------------
Model name: Cameron/BERT-mdgender-wizard | eval dataset: billsum
-------------------------
Model name: Cameron/BERT-mdgender-convai-binary | eval dataset: air_dialogue
-------------------------
Model name: Cameron/BERT-Jigsaw | eval dataset: yelp_review_full
-------------------------
Model name: Cameron/BERT-SBIC-targetcategory | eval dataset: yahoo_answers_topics
-------------------------
Model name: Cameron/BERT-mdgender-convai-ternary | eval dataset: yelp_review_full
-------------------------
Model name: Cameron/BERT-SBIC-offensive | eval dataset: pubmed_qa
-------------------------
Model name: Cameron/BERT-mdgender-wizard 

In [12]:
print(f"Found {len(toxic_files)} toxic results and {len(gender_files)} gender files")

Found 35 toxic results and 106 gender files


In [13]:
eval_files["conv_ai_3"]

{'toxic': [('BERT-Jigsaw',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-Jigsaw-00:40:33-conv_ai_3-train-eval.out'),
  ('BERT-SBIC-offensive',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-SBIC-offensive-00:40:33-conv_ai_3-train-eval.out')],
 'gender': [('BERT-mdgender-convai-ternary',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdgender-convai-ternary-00:40:41-conv_ai_3-train-eval.out'),
  ('BERT-mdgender-convai-binary',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdgender-convai-binary-00:40:37-conv_ai_3-train-eval.out'),
  ('BERT-rtgender-opgender-annotations',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-rtgender-opgender-annotations-00:40:32-conv_ai_3-train-eval.out'),
  ('BERT-eec-emotion',
   '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-eec-emotion-00:40:36-conv_ai_3-train-eval.out'),
  ('BERT-mdgender-wizard',
   '/scratch/amq259/nlu-s2021-final-project/results/C

In [14]:
from pprint import pprint
pprint(eval_files)

{'air_dialogue': {'gender': [('BERT-mdgender-convai-binary',
                              '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdgender-convai-binary-00:42:04-air_dialogue-train-eval.out'),
                             ('BERT-rtgender-opgender-annotations',
                              '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-rtgender-opgender-annotations-00:42:00-air_dialogue-train-eval.out'),
                             ('BERT-mdgender-wizard',
                              '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdgender-wizard-00:41:58-air_dialogue-train-eval.out'),
                             ('BERT-eec-emotion',
                              '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-eec-emotion-00:41:55-air_dialogue-train-eval.out'),
                             ('BERT-mdgender-convai-ternary',
                              '/scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdge

In [15]:
CACHE = {}
def get_cache(filename):
    if filename in CACHE:
        return CACHE[filename]
    return None

def store_cache(filename, data):
    CACHE[filename] = data
    
def cross_tab_gender_toxic(eval_file_split, eval_dataset_name, return_one = False):
    global CACHE
    CACHE = {}
    eval_crosstabs = []
    count = 0.0
    tot = len(eval_file_split["toxic"]) * len(eval_file_split["gender"])
    for toxic_tup in eval_file_split["toxic"]:
        toxic_model_name = toxic_tup[0]
        toxic_filename = toxic_tup[1] 
        for gender_tup in eval_file_split["gender"]:
            gender_model_name = gender_tup[0]
            gender_filename = gender_tup[1]
            
            # for using sbc and rt_gender for conv_ai
            _, _, combined = read_dfs(toxic_filename, 
                                      gender_filename,
                                     suffixes=("_" + toxic_model_name, "_" + gender_model_name))
            print(len(combined))
            pred_toxic = "predictions_" + toxic_model_name
            pred_gender = "predictions_" + gender_model_name
            
            print("relabeling toxic")
            s = get_cache(toxic_filename)
            if s is None:
                print(f"did not find {toxic_filename} in cache")
                p = partial(score, MODEL_RELABEL[toxic_model_name])
                s = combined[pred_toxic].swifter.apply(p)
                scores_col = f"scores_{toxic_model_name}"
                category_col = f"category_{toxic_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(toxic_filename, s)
            else:
                print(f"found toxic {toxic_filename} in cache")
            combined = combined.join(s)
            print("relabeling gender")
            s = get_cache(gender_filename)
            if s is None:
                print(f"did not find {gender_filename} in cache")
                p = partial(score, MODEL_RELABEL[gender_model_name])
                s = combined[pred_gender].swifter.apply(p)
                scores_col = f"scores_{gender_model_name}"
                category_col = f"category_{gender_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(gender_filename, s)
            else:
                print(f"found gender {gender_filename} in cache")
            combined = combined.join(s)
            
            
            eval_crosstabs.append({
                "toxic_model_name": toxic_model_name,
                "gender_model_name": gender_model_name,
                "toxic_attr": (f"scores_{toxic_model_name}", f"category_{toxic_model_name}"),
                "gender_attr": (f"scores_{gender_model_name}", f"category_{gender_model_name}"),
                "df": combined
            })
            count += 1.0
            print(f"Crossing {toxic_model_name} x {gender_model_name} for {eval_dataset_name}, finished {count/tot*100}%")
            if return_one:
                return eval_crosstabs
    return eval_crosstabs

In [16]:
eval_files.keys()

dict_keys(['wikipedia1114982', 'billsum', 'air_dialogue', 'yelp_review_full', 'yahoo_answers_topics', 'pubmed_qa', 'wikipedia564370', 'empathetic_dialogues', 'conv_ai_3', 'ted_talks_iwslt', 'wikipedia59654', 'wikipedia174925', 'tweet_eval'])

In [20]:
crosstabs = []
for eval_dataset_name, eval_f in eval_files.items():
    if eval_dataset_name != "wikipedia59654":
        continue
#     if eval_dataset_name == "air_dialogue" or eval_dataset_name == "empathetic_dialogues":
#         print(f'ignoring {eval_dataset_name} for now')
#         continue
    print(f"Analyzing {eval_dataset_name}")
    try:
        ct = cross_tab_gender_toxic(eval_f, eval_dataset_name, return_one=False)
        #pickle.dump((eval_dataset_name, ct), open(f"crosstabs-pickle-{eval_dataset_name}.p", "wb"))
        crosstabs.append((eval_dataset_name, ct))

    except Exception as e:
        print(e)
        raise 


Analyzing wikipedia59654
22842
relabeling toxic
did not find /scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-SBIC-offensive-17:12:58-wikipedia59654-train-eval.out in cache


Pandas Apply:   0%|          | 0/22842 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/22842 [00:00<?, ?it/s]

relabeling gender
did not find /scratch/amq259/nlu-s2021-final-project/results/Cameron-BERT-mdgender-wizard-17:07:40-wikipedia59654-train-eval.out in cache


Pandas Apply:   0%|          | 0/22842 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/22842 [00:00<?, ?it/s]

Crossing BERT-SBIC-offensive x BERT-mdgender-wizard for wikipedia59654, finished 8.333333333333332%


In [ ]:
pickle.dump(crosstabs, open("crosstabs-pickle.p", "wb"))

In [ ]:
len(crosstabs)

In [ ]:
def scale_and_generate(ct, dataset_name):
    toxic_model_name = ct["toxic_model_name"]
    gender_model_name = ct["gender_model_name"]
    toxic_score_col, toxic_category_col = ct["toxic_attr"]
    gender_score_col, gender_category_col = ct["gender_attr"]
    std_scaler = StandardScaler()
    df = ct["df"]
    # covariance 
    scores_df = df[[toxic_score_col, gender_score_col]]
    try:
        scores_df = pd.DataFrame(std_scaler.fit_transform(scores_df), columns=scores_df.columns)
    except Exception as e:
        print(df)
        print(f"FAILED TO GENERATE COVARIANCE FOR {dataset_name} WITH MODELS TOXIC: {toxic_model_name} GENDER: {gender_model_name}")
        return 
    cov = scores_df.cov()
    print(f"{dataset_name} | {toxic_category_col} | {gender_category_col}")
    print(cov)
    
    # contingency 
    contingency = pd.crosstab(df[toxic_category_col], df[gender_category_col])
    c, p, dof, expected = chi2_contingency(contingency)
    print(f"p-value is {p}")
    
    f, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(contingency, annot=True, fmt="d", cmap="YlGnBu").set_title(dataset_name)
    
    print("------------------------------")
    return contingency

In [ ]:
# for ct in crosstabs[0]:
#     scale_and_generate(ct, "conv_ai_3")

In [ ]:
# uncomment and change below to load a pickle and use that for analysis!
# crosstabs = pickle.load(open("crosstabs-pickle.p"))

In [ ]:
c = []
for eval_dataset_name, ct_list in crosstabs: 
    for ct in ct_list:
        contingency = scale_and_generate(ct, eval_dataset_name)
        c.append(contingency)